# Preparing

## Imports and Functions

In [1]:
import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import regex as re
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, Sequential, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping

### Normal

In [2]:
folder_path = '/Users/Felix/Google Drive/Felix/code/AgeRecognition/raw_data/Faces'
#folder_path = '/content/UTKFaces/UTKFaces/Faces'

In [3]:
def load_images_from_folder(folder_path,percent=100,gender='1'):
    filenames = [f for f in os.listdir(folder_path) if not f.startswith('.')]
    filenames.sort(key=lambda f: int(re.sub('\D', '', f)))
    images = []
    for filename in filenames:
        parsed_name = filename.split('_')
        if parsed_name[1] == gender:
            img = cv2.imread(os.path.join(folder_path,filename),cv2.IMREAD_UNCHANGED)

            scale_percent = percent # percent of original size
            width = int(img.shape[1] * scale_percent / 100)
            height = int(img.shape[0] * scale_percent / 100)
            dim = (width, height)

            # resize image
            resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            cv2.imshow("Resized image", resized)
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()
            if img is not None:
                img_colors = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
                images.append(img_colors)
    return images

def load_image_data_into_dataframe(folderpath, gender = '1'):

    filenames = [f for f in os.listdir(folder_path) if not f.startswith('.')]
    filenames.sort(key=lambda f: int(re.sub('\D', '', f)))
    df = pd.DataFrame()
    for filename in filenames:
        parsed_name = filename.split('_')
        #print(parsed_name[1])
        if parsed_name[1] == gender:
            df= df.append({'age' : int(parsed_name[0]),
                           'gender' : parsed_name[1],
                           'race' : parsed_name[2]}, ignore_index=True)
    df['gender'] = df['gender'].map({'0': 'male', '1': 'female'})
    df['race'] = df['race'].map({'0': 'White', '1': 'Black', '2': 'Asian', '3': 'Indian', '4': 'Others'})
    return df

def show_all_images_from_list(img_list,img_df):
    n=int(len(img_list)/5) +1 
    f = plt.figure(figsize=(25,n*5))
    for i,img in enumerate(img_list):
        f.add_subplot(n, 5, i + 1)
        plt.title(f'{img_df.iloc[i]}',
                  fontweight="bold",
                  fontsize=15)
        plt.imshow(img)
        f.tight_layout()

In [4]:
img_pic=load_images_from_folder(folder_path)

In [5]:
img_df = load_image_data_into_dataframe(folder_path)

## Targets into Age Bins of 5 (Baseline: 6%)

In [6]:
bins= [5*i for i in range(17)]
bins

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]

In [7]:
labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
img_df['age_group'] = pd.cut(img_df['age'], bins=bins, labels=labels)

In [8]:
img_df.tail(305)

,age,gender,race,age_group
10196,70.0,female,White,13
10197,70.0,female,White,13
10198,70.0,female,White,13
10199,70.0,female,White,13
10200,70.0,female,White,13
...,...,...,...,...
10496,80.0,female,Black,15
10497,80.0,female,Asian,15
10498,80.0,female,Asian,15
10499,80.0,female,Indian,15


In [9]:
img_df['age_group'].value_counts().sort_values()

15     118
14     138
13     184
12     206
9      304
11     317
8      323
10     328
2      335
1      451
3      590
7      747
6     1052
0     1183
4     1711
5     2514
Name: age_group, dtype: int64

## CNN Preperations Image Size: 100*100

In [10]:
img_pic=load_images_from_folder(folder_path,50)

In [11]:
X = img_pic
y = img_df['age_group']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train = np.array(X_train)
X_test =np.array(X_test)
y_train = np.array(y_train)
y_test =np.array(y_test)
X_train.shape

(9450, 100, 100, 3)

In [12]:
X_train = X_train/255 - 0.5
X_test = X_test/255 - 0.5
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [13]:
print(X_train.shape)
print(X_test.shape)

(9450, 100, 100, 3)
(1051, 100, 100, 3)


# CNN A

## CNN 2 (1 hidden with 10) 36,3%

In [14]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(10, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 8)         0         
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                184330    
_________________________________________________________________
dense_1 (Dense)              (None, 16)                176       
Total params: 184,898
Trainable params: 184,898
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
207/207 [==============================] - 34s 157ms/step - loss: 2.5962 - accuracy: 0.2806 - val_lo

## CNN 3 (1 hidden with 50 ) 36,1 %

In [15]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                921650    
_________________________________________________________________
dense_3 (Dense)              (None, 16)                816       
Total params: 922,858
Trainable params: 922,858
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
207/207 [==============================] - 44s 205ms/step - loss: 2.0974 - accuracy: 0.3150 - val_

## CNN 4 (1 hidden with 100 )  38,3%

In [16]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               1843300   
_________________________________________________________________
dense_5 (Dense)              (None, 16)                1616      
Total params: 1,845,308
Trainable params: 1,845,308
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
207/207 [==============================] - 40s 190ms/step - loss: 2.1450 - accuracy: 0.2958 - 

## CNN 5 (2 hidden with 100,50 )  38,6%

In [17]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Epoch 1/10
207/207 [==============================] - 46s 215ms/step - loss: 2.1359 - accuracy: 0.3054 - val_loss: 1.9535 - val_accuracy: 0.3330
Epoch 2/10
207/207 [==============================] - 39s 189ms/step - loss: 1.7842 - accuracy: 0.3802 - val_loss: 1.7999 - val_accuracy: 0.3714
Epoch 3/10
207/207 [==============================] - 45s 216ms/step - loss: 1.5577 - accuracy: 0.4435 - val_loss: 1.7522 - val_accuracy: 0.3728
Epoch 4/10
207/207 [==============================] - 43s 209ms/step - loss: 1.3790 - accuracy: 0.4933 - val_loss: 1.7632 - val_accuracy: 0.3672
Epoch 5/10
207/207 [==============================] - 30s 146ms/step - loss: 1.2141 - accuracy: 0.5601 - val_loss: 1.9071 - val_accuracy: 0.3668
Epoch 6/10
207/207 [==============================] - 30s 145ms/step - loss: 1.0406 - accuracy: 0.6180 - val_loss: 1.9274 - val_accuracy: 0.3619
Epoch 7/10
207/207 [==============================] - 31s 149ms/step - loss: 0.8791 - accuracy: 0.6821 - val_loss: 2.2066 - val_ac

## CNN 6 (2 hidden with 100,100 ) 39,8%

In [18]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Epoch 1/10
207/207 [==============================] - 37s 174ms/step - loss: 2.0979 - accuracy: 0.3043 - val_loss: 1.9066 - val_accuracy: 0.3513
Epoch 2/10
207/207 [==============================] - 31s 150ms/step - loss: 1.7408 - accuracy: 0.3794 - val_loss: 1.8070 - val_accuracy: 0.3683
Epoch 3/10
207/207 [==============================] - 32s 154ms/step - loss: 1.5464 - accuracy: 0.4382 - val_loss: 1.8022 - val_accuracy: 0.3467
Epoch 4/10
207/207 [==============================] - 31s 151ms/step - loss: 1.3556 - accuracy: 0.5029 - val_loss: 1.7679 - val_accuracy: 0.3901
Epoch 5/10
207/207 [==============================] - 34s 165ms/step - loss: 1.1901 - accuracy: 0.5664 - val_loss: 1.8961 - val_accuracy: 0.3481
Epoch 6/10
207/207 [==============================] - 33s 159ms/step - loss: 1.0173 - accuracy: 0.6310 - val_loss: 2.0127 - val_accuracy: 0.3711
Epoch 7/10
207/207 [==============================] - 32s 154ms/step - loss: 0.8460 - accuracy: 0.6981 - val_loss: 2.1624 - val_ac

## CNN 7 (3 hidden with 50,100,50 )  39,1%

In [19]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 50)                921650    
_________________________________________________________________
dense_13 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_14 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_15 (Dense)             (None, 16)               

## CNN 8 (1 hidden with 300 )  37,4%

In [20]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(300, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 300)               5529900   
_________________________________________________________________
dense_17 (Dense)             (None, 16)                4816      
Total params: 5,535,108
Trainable params: 5,535,108
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
207/207 [==============================] - 43s 200ms/step - loss: 2.1209 - accuracy: 0.3110 - 

## CNN 9 (2 hidden with 100,20 )  33,8%

In [21]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(20, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_18 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dense_19 (Dense)             (None, 20)                2020      
_________________________________________________________________
dense_20 (Dense)             (None, 16)                336       
Total params: 1,846,048
Trainable params: 1,846,048
Non-trainable params: 0
____________________________________________

## CNN 10 (3 hidden with 20, 50, 20 )  39,6%

In [22]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(20, activation='relu'))

### Second fully connected
model.add(layers.Dense(50, activation='relu'))

### Third fully connected
model.add(layers.Dense(20, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_21 (Dense)             (None, 20)                368660    
_________________________________________________________________
dense_22 (Dense)             (None, 50)                1050      
_________________________________________________________________
dense_23 (Dense)             (None, 20)                1020      
_________________________________________________________________
dense_24 (Dense)             (None, 16)               

# CNN B

## CNN 1 (3 hidden with 50,100,50 ) - (Epochs:100, Patience: 10)   37,4%
**High_Acc - low Val_Acc**

In [23]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))

model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_25 (Dense)             (None, 50)                921650    
_________________________________________________________________
dense_26 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_27 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_28 (Dense)             (None, 16)               

## CNN 1.2 - Dropout, Hidden Layer 50% => 37,7%

In [24]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())


### One fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.5))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.5))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.5))

model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_29 (Dense)             (None, 50)                921650    
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 100)               5100      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)             

## CNN 1.3 Dropout First and Hidden Layer 20%    34,8 %

In [25]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(Dropout(0.2, input_shape=(100,100,3,)))
model.add(layers.Conv2D(8, (4,4), activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.2))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.2))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.2))

### Last layer 
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_3 (Dropout)          (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_33 (Dense)             (None, 50)                921650    
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 100)             

## CNN 1.4 Dropout, Hidden Layer 40% => 36,5%

In [26]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.4))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.4))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.4))

### Last layer 
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_37 (Dense)             (None, 50)                921650    
_________________________________________________________________
dropout_7 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 100)               5100      
_________________________________________________________________
dropout_8 (Dropout)          (None, 100)             

## CNN 1.5 Dropout, Hidden Layer 30% => 36,2%

In [27]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.3))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.3))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.3))

### Last layer 
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_41 (Dense)             (None, 50)                921650    
_________________________________________________________________
dropout_10 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 100)               5100      
_________________________________________________________________
dropout_11 (Dropout)         (None, 100)             

## CNN 1.6 Dropout, Hidden Layer 20% => 38,5%

In [28]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.2))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.2))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.2))

### Last layer 
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_45 (Dense)             (None, 50)                921650    
_________________________________________________________________
dropout_13 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_46 (Dense)             (None, 100)               5100      
_________________________________________________________________
dropout_14 (Dropout)         (None, 100)             

# CNN C Something new (Copy from Cifar Classification) 40,3 %

In [30]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(100, 100, 3)))
#model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
#model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (2, 2), activation='relu', padding='same'))
#model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.3))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(16, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 100, 100, 32)      896       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 25, 25, 64)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 25, 25, 128)     

In [31]:
print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

0.40342530608177185


# CNN D Copy from Internet

In [32]:
model = Sequential()
model.add(Conv2D(input_shape=(100, 100, 3), filters=96, kernel_size=(7, 7), strides=(2, 2), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(LayerNormalization())
model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(LayerNormalization())
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(LayerNormalization())

model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=16, activation='softmax'))

print(model.summary())

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 47, 47, 96)        14208     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 23, 23, 96)        0         
_________________________________________________________________
layer_normalization (LayerNo (None, 23, 23, 96)        192       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 23, 23, 256)       614656    
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 11, 11, 256)       0         
_________________________________________________________________
layer_normalization_1 (Layer (None, 11, 11, 256)       512       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 11, 11, 256)     

KeyboardInterrupt: 